In [81]:
import pandas as pd
import pickle

In [82]:
with open("./raw/courses_with_syllabus_all.pickle", "rb") as f:
    data_dict = pickle.load(f)

In [83]:
df = pd.DataFrame.from_dict(data_dict)
df

,lmt_yn,nemo_yn,eng100,year,tutorial_yn,mooc_yn,isu_nm,courgrad_cd,cour_cd,rowid,...,cour_nm,dept_cd,cour_cls,drop_lmt_yn,time,apply_dept,exch_cor_yn,no_supervisor_yn,col_cd,detailed_info
0,Y,N,N,2024,N,N,전공선택,0136,ACEE123,1,...,건축시스템의이해,5204,00,N,3(3),,Y,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': [], '성적평가..."
1,Y,N,N,2024,N,N,전공선택,0136,ACEE219,2,...,환경화학(영강),5204,00,N,3(3),[*에코업track],N,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': ['이론강의', ..."
2,Y,N,N,2024,N,N,전공선택,0136,ACEE227,3,...,유체역학및실험,5204,01,N,3(4),[*에코업track],N,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': [], '성적평가..."
3,Y,N,N,2024,N,N,전공선택,0136,ACEE227,4,...,유체역학및실험(영강),5204,02,N,3(4),[*에코업track],N,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': ['이론강의', ..."
4,Y,N,N,2024,N,N,전공선택,0136,ACEE228,5,...,구조역학I(영강),5204,00,N,3(4),[*에코업track],N,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': [], '성적평가..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53400,N,N,N,2017,N,N,전공필수,0136,STEP410,2936,...,현장실습V,5965,00,N,3(0),,Y,N,9999,"{'수업유형': None, '특별유형': ['현장실습'], '수업구성요소': [],..."
53401,N,N,N,2017,N,N,전공필수,0136,STEP411,2937,...,현장실습VI,5965,00,N,3(0),,Y,N,9999,"{'수업유형': None, '특별유형': ['현장실습'], '수업구성요소': [],..."
53402,N,N,N,2017,N,N,전공필수,0136,STEP412,2938,...,현장실습VII,5965,00,N,3(0),,Y,N,9999,"{'수업유형': None, '특별유형': ['현장실습'], '수업구성요소': [],..."
53403,N,N,N,2017,N,N,전공필수,0136,STEP413,2939,...,현장실습VIII,5965,00,N,3(0),,Y,N,9999,"{'수업유형': None, '특별유형': ['현장실습'], '수업구성요소': [],..."


해당 데이터셋에서 하나의 필드에 값이 하나로 결정돼있는 필드들이 있습니다. (waiting_yn, col_cd...) </br>
해당 필드들을 날려줍니다.

In [84]:
unique_counts = {}
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, dict)).any():
        continue
    n_unique = df[col].nunique()
    unique_counts[col] = n_unique
unique_one_cols = [col for col, count in unique_counts.items() if count == 1]
df.drop(columns=unique_one_cols, axis=1, inplace=True)

영향이 없거나 필요 없는 필드가 존재합니다 (rowid, flexible_to_dt...) </br>
또한 params는 학수번호와 분반으로 유도가 되는 필드값이므로 같이 날립니다.

In [85]:
df.drop(columns=["rowid", "params", "flexible_to_dt", "flexible_fr_dt", "cour_div"], axis=1, inplace=True)

Y/N 값을 불리언 값으로 변환합니다.

In [86]:
yn_columns = [col for col in df.columns if col.lower().endswith('yn')]
yn_columns += ["eng100"]
for yn_column in yn_columns:
    df[yn_column] = df[yn_column].map({'Y': True, 'N': False}).astype(bool)

|   | time |
|---|------|
| 0 | 3(3) |
| 1 | 3(3) |
| 2 | 3(4) |
| 3 | 3(4) |

time 필드의 경우 `학점(강의 시간)`으로 정의 되어 있어 둘을 각각의 필드로 구분돼야합니다. </br>
학점의 경우 credit 필드로 이미 정의 돼 있으므로 time 필드 내에서 강의 시간만 따로 추출하여 값을 대체합니다.

In [87]:
def get_time(value:str):
    s = value.find("(")
    e = value.find(")")
    return value[s + 1:e]
df["time"] = df["time"].apply(get_time).astype(float)

year 필드 Object형식을 숫자값으로 변환합니다

In [88]:
df['year'] = df['year'].astype(int)

In [89]:
df["isu_nm"].unique()

array(['전공선택', '전공필수', '학문의기초', '교양', '교직(비사대)', '군사학', '학부공통', '교직',
       '평생교육사'], dtype=object)

isu_nm 필드는 과목의 이수구분으로 순서가 없는 범주형 데이터이므로 </br>
one-hot 인코딩을 통해 `전공선택`, `전공필수`, `학문의기초`... 등 각 필드를 새로 만들어 불리언 값을 배정합니다.

In [90]:
df = pd.get_dummies(df, columns=['isu_nm'], drop_first=True, prefix='이수구분', prefix_sep='.')

`time_room` 필드는 수업요일(교시) 장소 순으로 들어가 있으므로 유의미한 정보를 추출해내기 위해 one hot 인코딩을 진행합니다. </br>
ㅇ요일 ㅇ교시 수업 필드를 만들어 해당 교시에 수업이 있으면 장소를 없으면 `Nan`을 채워넣습니다</br>
|   | 월요일 1교시 | 월요일 2교시 | ... |
|---|------------|-------------|----|
| 0 | 교육관 313호      |   Nan       | ... |
| 1 | Nan     |   미정        | ... |


In [91]:

import re
import pandas as pd


def extract_partial_matches(text):
    pattern = r'([월화수목금토일])\((\d+)(?:-(\d+))?\)\s*(.*?)(?=<br>|$)'
    matches = re.findall(pattern, text)

    results = []
    for match in matches:
        week_of_day = match[0]
        start_period = int(match[1])
        end_period = int(match[2]) if match[2] else start_period
        room = match[3].strip()
        results.append((week_of_day, start_period, end_period, room))

    return results


# 요일별 수업 여부를 담을 컬럼들
weekdays = ['월', '화', '수', '목', '금', '토', '일']
weekday_columns = [f"{day}요일" for day in weekdays]

# 각 row 처리
expanded_data = []
for schedule_entries in df["time_room"]:
    record = {col: False for col in weekday_columns}  # 모든 요일을 False로 초기화

    for day, start_period, end_period, room in extract_partial_matches(schedule_entries):
        day_column = f"{day}요일"
        record[day_column] = True  # 해당 요일에 수업이 있으면 True

    expanded_data.append(record)

# DataFrame으로 변환하고 기존 DataFrame과 결합
expanded_df = pd.DataFrame(expanded_data)
df = pd.concat([df, expanded_df], axis=1)
df.drop(columns=["time_room"], axis=1, inplace=True)

In [92]:
filed_name_dict = {
    'year': '년도',
    'term': '학기',
    'cour_cd': '학수번호',
    'cour_cls': '분반',
    'isu_nm': '이수구분',
    'department': '개설학과',
    'cour_nm': '교과목명',
    'prof_nm': '담당교수',
    'time_room': '강의교시',
    'apply_dept': '학과제한',
    'absolute_yn': '상대평가',
    'lmt_yn': '인원제한',
    'exch_cor_yn': '교환학생',
    'attend_free_yn': '출석확인자율화',
    'no_supervisor_yn': '무감독시험',
    'flexible_term': '유연학기',
    'mooc_yn': 'MOOC',
    'flipped_class_yn': 'Flipped',
    'nemo_yn': 'NeMo',
    'eng100': '영강',
    'drop_lmt_yn': '수강포기제한',
    'time' : '강의시간',
    'credit' : '학점',
    'tutorial_yn': "튜토리얼"
}
df.rename(columns=filed_name_dict, inplace=True)

In [93]:
df

,인원제한,NeMo,영강,년도,튜토리얼,MOOC,학수번호,학기,학점,개설학과,...,이수구분.평생교육사,이수구분.학문의기초,이수구분.학부공통,월요일,화요일,수요일,목요일,금요일,토요일,일요일
0,True,False,False,2024,False,False,ACEE123,1R,3.0,건축사회환경공학부,...,False,False,False,True,False,False,False,False,False,False
1,True,False,False,2024,False,False,ACEE219,1R,3.0,건축사회환경공학부,...,False,False,False,True,False,True,False,False,False,False
2,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,True,False,True,False,False,False
3,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,True,False,True,False,False,False
4,True,False,False,2024,False,False,ACEE228,1R,3.0,건축사회환경공학부,...,False,False,False,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53400,False,False,False,2017,False,False,STEP410,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,False,False,False
53401,False,False,False,2017,False,False,STEP411,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,False,False,False
53402,False,False,False,2017,False,False,STEP412,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,False,False,False
53403,False,False,False,2017,False,False,STEP413,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,False,False,False


In [94]:
df["수업유형"] = df["detailed_info"].apply(lambda x: x.get("수업유형", None))
one_hot = pd.get_dummies(df["수업유형"], prefix="수업유형")
df.drop(columns=["수업유형"], axis=1)
df = pd.concat([df, one_hot], axis=1)

In [95]:
df

,인원제한,NeMo,영강,년도,튜토리얼,MOOC,학수번호,학기,학점,개설학과,...,금요일,토요일,일요일,수업유형,수업유형_대면,수업유형_병행,수업유형_원격,수업유형_원격(녹화),수업유형_원격(실시간),수업유형_혼합
0,True,False,False,2024,False,False,ACEE123,1R,3.0,건축사회환경공학부,...,False,False,False,대면,True,False,False,False,False,False
1,True,False,False,2024,False,False,ACEE219,1R,3.0,건축사회환경공학부,...,False,False,False,대면,True,False,False,False,False,False
2,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,대면,True,False,False,False,False,False
3,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,대면,True,False,False,False,False,False
4,True,False,False,2024,False,False,ACEE228,1R,3.0,건축사회환경공학부,...,False,False,False,대면,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53400,False,False,False,2017,False,False,STEP410,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,None,False,False,False,False,False,False
53401,False,False,False,2017,False,False,STEP411,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,None,False,False,False,False,False,False
53402,False,False,False,2017,False,False,STEP412,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,None,False,False,False,False,False,False
53403,False,False,False,2017,False,False,STEP413,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,None,False,False,False,False,False,False


In [96]:
df_expanded = pd.json_normalize(df["detailed_info"])

In [97]:
df_expanded

,수업유형,특별유형,수업구성요소,성적평가,평가방법.출석,평가방법.과제,평가방법.중간고사,평가방법.기말고사,평가방법.합계,평가방법.참여,...,평가방법.연극감상문,평가방법.수시 - 작품 읽기,평가방법.참여도- 발표 및 토론,평가방법.독서감상문,평가방법.팀별 사업계획 평가,평가방법.참여 적극성 평가,평가방법.온라인 강의수강,평가방법.Homework (6 times),평가방법.Quizzes (2 times),평가방법.4 Homework Assignments
0,대면,[],[],[절대평가],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,대면,[],"[이론강의, 퀴즈, Q&A]",[절대평가],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,대면,[],[],[절대평가],10 %,10 %,40 %,40 %,100%,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,대면,[],"[이론강의, 실험]",[절대평가],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,대면,[],[],[절대평가],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53400,None,[현장실습],[],"[절대평가, P/F]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53401,None,[현장실습],[],"[절대평가, P/F]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53402,None,[현장실습],[],"[절대평가, P/F]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53403,None,[현장실습],[],"[절대평가, P/F]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# 특별유형 one-hot
special = df_expanded[['특별유형']].explode('특별유형')
special_dummies = pd.get_dummies(special['특별유형'], prefix='특별유형')
special_ohe = special_dummies.groupby(special.index).sum()
special_ohe = special_ohe.reindex(df_expanded.index, fill_value=0)

# 수업구성요소 one-hot
course = df_expanded[['수업구성요소']].explode('수업구성요소')
course_dummies = pd.get_dummies(course['수업구성요소'], prefix='수업구성요소')
course_ohe = course_dummies.groupby(course.index).sum()
course_ohe = course_ohe.reindex(df_expanded.index, fill_value=0)

# 성적평가 one-hot
grade = df_expanded[['성적평가']].explode('성적평가')
grade_dummies = pd.get_dummies(grade['성적평가'], prefix='성적평가')
grade_ohe = grade_dummies.groupby(grade.index).sum()
grade_ohe = grade_ohe.reindex(df_expanded.index, fill_value=0)

special_ohe = special_ohe.astype(bool)
course_ohe = course_ohe.astype(bool)
grade_ohe = grade_ohe.astype(bool)

df = pd.concat([df, special_ohe, course_ohe, grade_ohe], axis=1)

In [99]:
df

,인원제한,NeMo,영강,년도,튜토리얼,MOOC,학수번호,학기,학점,개설학과,...,수업구성요소_체험,수업구성요소_퀴즈,수업구성요소_토론,수업구성요소_특강,수업구성요소_포럼,수업구성요소_프로젝트,수업구성요소_협동학습,성적평가_P/F,성적평가_상대평가,성적평가_절대평가
0,True,False,False,2024,False,False,ACEE123,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
1,True,False,False,2024,False,False,ACEE219,1R,3.0,건축사회환경공학부,...,False,True,False,False,False,False,False,False,False,True
2,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
3,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
4,True,False,False,2024,False,False,ACEE228,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53400,False,False,False,2017,False,False,STEP410,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,True,False,True
53401,False,False,False,2017,False,False,STEP411,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,True,False,True
53402,False,False,False,2017,False,False,STEP412,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,True,False,True
53403,False,False,False,2017,False,False,STEP413,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,True,False,True


In [100]:
df.drop(columns=["detailed_info", "flexible_school_yn", "waiting_yn", "dept_cd", "학과제한"], inplace=True)

In [101]:
df

,인원제한,NeMo,영강,년도,튜토리얼,MOOC,학수번호,학기,학점,개설학과,...,수업구성요소_체험,수업구성요소_퀴즈,수업구성요소_토론,수업구성요소_특강,수업구성요소_포럼,수업구성요소_프로젝트,수업구성요소_협동학습,성적평가_P/F,성적평가_상대평가,성적평가_절대평가
0,True,False,False,2024,False,False,ACEE123,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
1,True,False,False,2024,False,False,ACEE219,1R,3.0,건축사회환경공학부,...,False,True,False,False,False,False,False,False,False,True
2,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
3,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
4,True,False,False,2024,False,False,ACEE228,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53400,False,False,False,2017,False,False,STEP410,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,True,False,True
53401,False,False,False,2017,False,False,STEP411,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,True,False,True
53402,False,False,False,2017,False,False,STEP412,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,True,False,True
53403,False,False,False,2017,False,False,STEP413,2R,3.0,소프트웨어벤처융합전공,...,False,False,False,False,False,False,False,True,False,True


In [102]:
df.to_csv("syllabus_preprocessing.csv", encoding="utf-8")